<a href="https://colab.research.google.com/github/karlmaji/pytorch_learning/blob/master/Swin_transformer%E8%AE%BA%E6%96%87%E5%A4%8D%E7%8E%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Step1 img to patch

In [16]:
# patch_depth = patchsize *patchsize *input_channel
# patch_depth= 12
# model_dim = 8
# weight = torch.randn(12,8)


# 方法1：使用unfold 方法
def img2patch(img,patchsize,weight):
  patch = F.unfold(img,kernel_size=patchsize,
                   stride=patchsize).transpose(-1,-2) 
                   #[bs,patch_num,patch_depth]

  patch_embedding = patch @ weight #weight.shape:[patch_depth,model_dim]

  # print(patch_embedding.shape) #[bs,num_patch,model_dim]
  
  return patch_embedding

# 方法2:使用卷积方法
def img2patch_conv(img,patchsize,model_dim):
  bs,i_c,h,w =img.shape

  #实际建立model时 先定义卷积层在调用
  layer= nn.Conv2d(3,model_dim,kernel_size=patchsize,stride=patchsize)
  #output:[bs,model_dim,h,w]

  patch_embedding=layer(img).reshape(bs,model_dim,-1).transpose(-1,-2)
  # print(patch_embedding.shape)


# img=torch.randn(2,3,224,224)
# img2patch(img,2,weight)
# img2patch_conv(img,2,model_dim)


# Step2 构建MultiHeadSelfAttention

In [69]:
class MultiHeadSelfAttention(nn.Module):
  def __init__(self,model_dim,num_head):
    super(MultiHeadSelfAttention, self).__init__()
    self.headnum = num_head
    self.proj_Linear = nn.Linear(model_dim,3*model_dim)
    self.final_Linear = nn.Linear(model_dim,model_dim)

  def forward(self,input_,with_mask):
    bs,seq_len,model_dim = input_.shape

    num_head = self.headnum
    head_dim = model_dim // num_head

    proj_output = self.proj_Linear(input_) # [bs, seqlen, 3*model_dim] 

    proj_output = proj_output.reshape(bs,seq_len,3,\
                    num_head,head_dim).permute(2,0,3,1,4) 
                  #output:[3,bs,num_head,seq_len,head_dim]

    q, k, v = proj_output.reshape(3,bs*num_head,seq_len,head_dim)[:]
    #q,k,v .shape= [bs*num_head,seq_len,head_dim]
    
    if with_mask==None:
      atten_prob=F.softmax(torch.bmm(q,k.transpose(-1,-2)/torch.sqrt(torch.tensor(head_dim))),dim=-1)


    output = torch.bmm(atten_prob, v) # [bs*num_head, seqlen, head_dim]
    output = output.reshape(bs, num_head, seq_len, head_dim).transpose(1, 2) #[bs, seqlen, num_head, head_dim]
    output = output.reshape(bs, seq_len, model_dim)

    output = self.final_Linear(output)
    return output


